In [ ]:
num = 1

In [ ]:
!pip install -U FlagEmbedding
!pip install einops
!pip install gdown

In [ ]:
import gdown
import zipfile
import os

# 1. Tải file .zip từ Google Drive
url = 'https://drive.google.com/uc?id=1pTDobti0RqdEJpmrboLSsz-JKtyO8Da2'  # Thay bằng ID của file zip
output_zip = 'bi_encoder_search.zip'  # Đặt tên file .zip sau khi tải về

# Tải file .zip
gdown.download(url, output_zip, quiet=False)

# 2. Giải nén file .zip
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('extracted_files')  # Thư mục để giải nén (nếu không tồn tại, nó sẽ tự tạo)

# Kiểm tra kết quả
print("File đã được giải nén vào thư mục 'extracted_files'")


In [ ]:
import gdown

# URL tải file từ Google Drive
url = 'https://drive.google.com/uc?id=1PaTajVGgFkYLP45UAmjWSgwL6Yi3-w7g'
output = '/kaggle/working/template_chunking_flat.json'  # Đặt tên file bạn muốn lưu
gdown.download(url, output, quiet=False)

In [ ]:
import gdown

# URL tải file từ Google Drive
url = 'https://drive.google.com/uc?id=1wy8i6qAVWgvD4vzyyY7_3_0aj0J4gz5z' 
csv = '/kaggle/working/test_data.csv'  

# Tải file .json
gdown.download(url, csv, quiet=False)

print("File JSON đã được tải xuống.")


In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

In [ ]:
#load test_data
import pandas as pd

data = pd.read_csv('/kaggle/working/test_data.csv') 
questions = data['question'].tolist()
qids = data['qid'].tolist()


In [ ]:
import json


# Load the text data
with open('/kaggle/working/template_chunking_flat.json', 'r', encoding='utf-8') as f:
    text_data = json.load(f)
    
# Tải file
text_dict = {str(item['stt']): item['text'] for item in text_data}
stt_dict = {item['text']: item['stt'] for item in text_data}


In [ ]:
# Load the qid and ids data
qid_id_data = []
with open(f'/kaggle/working/extracted_files/output_part_{num}.txt', 'r') as file:
    for line in file:
        qid_and_ids = line.strip().split()
        qid = qid_and_ids[0]
        ids = qid_and_ids[1:]

        question = data[data['qid'] == int(qid)]['question'].values[0]

        answers = [text_dict.get(id, '') for id in ids] 
        qid_id_data.append((qid, question, answers))


In [ ]:
from tqdm import tqdm

# Rerank the answers
rerank_results = []
for qid, question, answers in tqdm(qid_id_data, desc="Processing QID data"):
    # Create question-answer pairs
    question_answer_pairs = [(question, answer) for answer in answers if answer]  # Filter out empty answers

    # Skip if no valid answers
    if not question_answer_pairs:
        continue

    # Compute similarity scores
    pairs = [(question, answer) for question, answer in question_answer_pairs]
    scores = reranker.compute_score(pairs, normalize=True)

    # Rank the answers based on similarity scores
    ranked_answers = [(answer, score) for (answer, score) in zip(answers, scores)]
    ranked_answers.sort(key=lambda x: x[1], reverse=True)  # Sort by similarity score in descending order

    # Append the ranked answers with their corresponding stt values
    ranked_stts = [stt_dict[answer] for answer, _ in ranked_answers]
    rerank_results.append((qid, ranked_stts))  # Save the sorted stt values

# Save the reranked results to a file
with open('reranked_output.txt', 'w') as f:
    for qid, ranked_stts in tqdm(rerank_results, desc="Writing results to file"):
        # Create the result line for each qid
        result_line = f"{qid} " + " ".join(map(str, ranked_stts))  # Combine qid and stt values
        f.write(result_line + "\n")

In [ ]:
import json

# Đọc file JSON chứa thông tin
with open('/kaggle/working/template_chunking_flat.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Tạo một từ điển để tra cứu stt -> infor_id
stt_to_infor_id = {str(item['stt']): int(item['infor_id']) for item in json_data}

# Đọc file reranked_output.txt và thay thế id bằng infor_id (kiểu int)
reranked_results = []
with open('reranked_output.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        if not parts:
            continue
        qid = parts[0]
        ids = parts[1:]
        # Chỉ lấy top 10 IDs
        top_10_ids = ids[:10]
        # Thay thế id bằng infor_id và đảm bảo infor_id là int
        infor_ids = [stt_to_infor_id.get(i, i) for i in top_10_ids]
        reranked_results.append(f"{qid} " + " ".join(map(str, infor_ids)))

with open(f'reranked_infor_output_part{num}.txt', 'w', encoding='utf-8') as f:
    for result in reranked_results:
        f.write(result + "\n")

In [ ]:
import zipfile

# Đặt tên file đầu vào và đầu ra
input_file = f'reranked_infor_output_part{num}.txt'  # Thay bằng tên file của bạn
output_zip_file = f'reranked_infor_output_part{num}.zip'  # Tên file zip đầu ra

# Nén file .txt vào .zip
with zipfile.ZipFile(output_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(input_file, arcname=input_file.split('/')[-1])  # Đặt tên file trong zip

print(f"Đã nén file {input_file} thành {output_zip_file}")
